In [345]:
from pathlib import Path

audio_dir = Path("librispeech/audio")
audio_ext = ".flac"
paths = list(audio_dir.rglob(f"**/*{audio_ext}"))

In [485]:
import pandas as pd
import torchaudio
import torch

align_dir = Path("librispeech/alignments")
align_df = pd.read_csv(align_dir / "alignments.csv")

model_name = "hubert_base"
layer = 12


In [ ]:
try:
    bundle = getattr(torchaudio.pipelines, model_name.upper())
except AttributeError:
    raise ValueError(f"Invalid model name: {model_name}")

model = bundle.get_model()
model.eval()

In [347]:
def cut_encoding(
    encoding: torch.Tensor, word_boundaries: list[float], hop_ms: int = 20
) -> torch.Tensor:
    hop_size = hop_ms / 1000  # seconds per frame
    start_frame = int(word_boundaries[0] / hop_size)
    end_frame = int(word_boundaries[1] / hop_size)
    return encoding[start_frame:end_frame]

In [349]:
from typing import Tuple

import numba
import numpy as np
import scipy.spatial.distance as distance


def segment(
    sequence: np.ndarray, codebook: np.ndarray, gamma: float
) -> Tuple[np.ndarray, np.ndarray]:
    """Group speech representations into phone-like segments.

    Args:
        sequence (NDArray): speech representations of shape (T, D) where T is the number of frames and D is the feature dimension.
        codebook (NDArray): cluster centriods of the discrete units of shape (K, D) where K is the number of codes.
        gamma float: Duration regularizer weight. Larger values result in a coarser segmentation.

    Returns:
        NDArray[int]: list of discrete units representing each segment sound types of shape (N,).
        NDArray[int]: list of segment boundaries of shape (N+1,).
    """
    dists = distance.cdist(sequence, codebook).astype(np.float32)
    alpha, P = _segment(dists, gamma)
    return _backtrack(alpha, P)


@numba.njit()
def _segment(dists, gamma):
    T, K = dists.shape

    alpha = np.zeros(T + 1, dtype=np.float32)
    P = np.zeros((T + 1, 2), dtype=np.int32)
    D = np.zeros((T, T, K), dtype=np.float32)

    for t in range(T):
        for k in range(K):
            D[t, t, k] = dists[t, k]
    for t in range(T):
        for s in range(t + 1, T):
            D[t, s, :] = D[t, s - 1, :] + dists[s, :] - gamma

    for t in range(T):
        alpha[t + 1] = np.inf
        for s in range(t + 1):
            k = np.argmin(D[s, t, :])
            alpha_min = alpha[s] + D[s, t, k]
            if alpha_min < alpha[t + 1]:
                P[t + 1, :] = s, k
                alpha[t + 1] = alpha_min
    return alpha, P


@numba.njit()
def _backtrack(alpha, P):
    rhs = len(alpha) - 1
    segments = []
    boundaries = [rhs]
    while rhs != 0:
        lhs, code = P[rhs, :]
        segments.append(code)
        boundaries.append(lhs)
        rhs = lhs
    segments.reverse()
    boundaries.reverse()
    return segments

In [350]:
import joblib

kmeans_path = f"models/kmeans_{model_name}_layer{layer}_k100.pkl"
kmeans = joblib.load(kmeans_path)

In [351]:
import numpy as np
import itertools
import editdistance


def normalized_edit_distance(sequences):
    pairs = list(itertools.combinations(sequences, 2))
    if not pairs:
        return 0.0

    dists = []
    for a, b in pairs:
        dist = editdistance.eval(a, b)
        norm = dist / max(len(a), len(b))
        dists.append(norm)

    return np.mean(dists)


In [ ]:
gamma = 1.0
feat_paths = list(
    Path(f"my-features/{model_name.upper()}/{layer}/gamma{gamma}").rglob("*.npy")
)
print(f"Found {len(feat_paths)} feature files.")

Found 65840 feature files.


In [484]:
codes = []
phn_labels = []
word_labels = []
for path in paths:
    wav_df = align_df[align_df["filename"] == path.stem]

    waveform, sr_loaded = torchaudio.load(str(path))
    if sr_loaded != 16000:
        waveform = torchaudio.functional.resample(waveform, sr_loaded, 16000)

    with torch.inference_mode():
        features, _ = model.extract_features(waveform, num_layers=layer)
        encoding = features[layer - 1].squeeze().cpu().numpy()

    for w in range(1, max(wav_df["word_id"]) + 1):
        word_df = wav_df[wav_df["word_id"] == w]

        text = str(word_df["text"].values[0])
        phones = tuple(str(word_df["phones"].values[0]).split(","))

        start = float(word_df["word_start"].values[0])
        end = float(word_df["word_end"].values[0])
        word_encoding = cut_encoding(encoding, [start, end])
        if len(word_encoding) == 0:
            continue
        word_codes = kmeans.predict(word_encoding)
        phn_labels.append(phones)
        word_labels.append(text)

        codes.append(segment(word_encoding, kmeans.cluster_centers_, 1.0))

        # if len(codes) >= 1500:
        #     break

        if len(codes) % 100 == 0:
            print(f"{len(codes)}")

    # if len(codes) >= 1500:
    #     break


100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18200
18300
18400
1850

KeyboardInterrupt: 

In [489]:
from tqdm import tqdm

codes = []
phn_labels = []
word_labels = []
for path in tqdm(feat_paths, desc="Processing features"):
    filename = path.stem.split("_")[0]
    w = int(path.stem.split("_")[-1])
    wav_df = align_df[align_df["filename"] == filename]

    word_df = wav_df[wav_df["word_id"] == w]

    text = str(word_df["text"].values[0])
    phones = tuple(str(word_df["phones"].values[0]).split(","))

    phn_labels.append(phones)
    word_labels.append(text)

    codes.append(np.load(path))

Processing features: 100%|██████████| 65840/65840 [04:12<00:00, 260.41it/s]


In [543]:
sil = [
    37,
    86,
    13,
    46,
    54,
    19,
    92,
    19,
    16,
    60,
    85,
    6,
    78,
    71,
    89,
    55,
    3,
    91,
    65,
    27,
    22,
    5,
]
cluster = []
dirty_cluster = []
for i, c in enumerate(codes):
    phones = [phn for phn in phn_labels[i]]

    # phones = [phn for phn in phn_labels[i] if phn not in ["spn", "sil", "sp", ""]]

    if len(c) > 0 and word_labels[i] == "can":
        clean_codes = [int(c) for c in codes[i] if c not in sil]
        cluster.append(clean_codes)
        dirty_cluster.append(codes[i])
        print(f"{'-'.join(phones):<5} | {clean_codes}")

print(f"Normalised edit distance (clean):  {normalized_edit_distance(cluster):.3f}")
print(f"Normalised edit distance:  {normalized_edit_distance(dirty_cluster):.3f}")


K-AH0-N | [73, 29, 35, 43, 93]
K-AH0-N | [41, 73, 52, 29, 75, 59]
K-AH0-N | [73, 52, 29, 75]
K-AE1-N | [41, 73, 42, 4, 70, 99, 53]
K-AE1-N | [0, 73, 42, 4, 29, 35, 43]
K-AH0-N | [41, 0, 73, 42, 29, 35, 43]
K-AH0-N | [0, 73, 42, 29, 35, 93]
K-AH0-N | [93, 73, 42, 29, 35]
K-AH0-N | [41, 73, 29, 35, 43]
K-AH0-N | [73, 29, 35]
K-AE1-N | [73, 42, 4, 29, 35, 36]
K-AH0-N | [41, 73, 42, 29, 35, 43]
K-AH0-N | [41, 0, 73, 52, 29, 35, 43]
K-AE1-N | [73, 42, 70, 99]
K-AH0-N | [41, 0, 73, 52, 29, 35, 43]
K-AH0-N | [73, 29, 35, 43]
K-AE1-N | [73, 42, 29, 35, 51]
K-AH0-N | [41, 73, 52, 29, 75, 59]
K-AH0-N | [73, 42, 29, 75]
K-AH0-N | [73, 8]
K-AH0-N | [0, 73, 42, 29, 75]
K-AH0-N | [41, 73, 29, 35]
K-AH0-N | [73, 52, 29, 35, 51]
K-AH0-N | [0, 73, 42, 29, 35, 59]
K-AH0-N | [41, 73, 29, 35, 18]
K-AH0-N | [41, 73, 29, 35]
K-AH0-N | [41, 73, 52, 29, 35]
K-AH0-N | [73, 29]
K-AE1-N | [41, 73, 42, 52, 29, 75, 59]
K-AH0-N | [41, 73, 42, 94, 67, 43]
K-AE1-N | [93, 73, 42, 29, 75]
K-AH0-N | [93, 73, 52, 29, 35,

In [546]:
import numpy as np


def calc_edit_distance(a, b):
    dist = editdistance.eval(a, b)
    norm = dist / max(len(a), len(b)) if len(a) > 0 and len(b) > 0 else 1.0

    return norm

In [547]:
subset_codes = codes[0 : int(len(codes) / 4)]
subset_phn_labels = phn_labels[0 : int(len(phn_labels) / 4)]
subset_word_labels = word_labels[0 : int(len(word_labels) / 4)]

In [ ]:
vals = []
rows = []
cols = []
for i in tqdm(range(len(subset_codes)), desc="Calculating edit distances"):
    for j in range(i, len(subset_codes)):
        vals.append(calc_edit_distance(subset_codes[i], subset_codes[j]))
        rows.append(i)
        cols.append(j)

Calculating edit distances:   1%|          | 176/16460 [00:05<08:58, 30.23it/s]

In [469]:
print(np.mean([d for d in vals if d < 1.0]))

0.8242965211138203


In [470]:
rows = np.array(rows)
cols = np.array(cols)
vals = np.array(vals)

In [471]:
import igraph as ig

g = ig.Graph()
g.add_vertices(len(codes))
mask = vals < 0.55
edges = list(zip(rows[mask], cols[mask]))
weights = vals[mask].astype(float)
weights = np.where(weights > 0, weights, 1e-10).tolist()

if edges:
    g.add_edges(edges)
    g.es[-len(weights) :].set_attribute_values("weight", weights)

In [482]:
import leidenalg as la

partition = la.find_partition(
    g,
    la.CPMVertexPartition,
    resolution_parameter=0.13,
    weights="weight",
    seed=42,
)

In [ ]:
print(f"Found {len(partition)} clusters, should be {len(set(subset_word_labels))}")
dists = []
for row in partition:
    clust = []
    words = []
    for el in row:
        phones = [
            phn for phn in subset_phn_labels[el] if phn not in ["spn", "sil", "sp", ""]
        ]
        word = subset_word_labels[el]
        words.append(word)
        phones = "-".join(phones)

        if phones:
            clust.append(phones)

    if clust:
        # print(f"{','.join([f' {phn}[{wrd}] ' for phn, wrd in zip(clust, words)])}\n")

        dist = normalized_edit_distance(clust)
        dists.append(dist)
        print(f"{dist * 100:<6.3f}% | {', '.join(words)}")

print(f"{np.mean(dists) * 100:.3f}%")

Found 491 clusters, should be 495
20.423% | the, the, the, the, the, the, the, the, the, there, the, the, the, the, the, the, there, they're, the, the, that, the, the, the, the, that, the, the, the, the, than, the, the, the, the, the, them, the, the, the, the, the, the, the, the, the, the, the, the, that, the, the, the, the
34.466% | was, was, were, with, was, was, was, which, was, was, was, was, was, with, was, we, was, was, was, was, was, with, was, was, what, would, was, would, what, was, what, was, were, with, was, which, with, way, way
36.459% | i, i, i, i, i, or, i, i, an, i, i, i, i, it, i, and, a, on, i, i, it, i, i, i, i, i, i, i, and, i, or
26.903% | was, as, was, is, is, was, is, as, is, is, is, as, is, is, has, is, is, is, is, is, is, is, is
9.697 % | to, to, to, to, to, to, to, to, to, to, to, to, to, to, to, to, to, to, to, to, to, to
40.017% | and, been, an, and, an, in, and, and, in, in, in, in, in, and, in, in, and, on, in
10.292% | of, of, of, of, of, of, of, of, of, 